In [1]:
# =========================
# Cell 1 — ENV (MUST be first, before torch/transformers)
# =========================
from ft_pipeline.env import apply_env
apply_env()

import os
import gc
import torch

torch.backends.cuda.matmul.allow_tf32 = True
torch.set_float32_matmul_precision("high")

print("CUDA available:", torch.cuda.is_available())
print("CUDA device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else None)



import gc, torch
print("allocated:", torch.cuda.memory_allocated()/1024**2, "MB")
print("reserved:",  torch.cuda.memory_reserved()/1024**2, "MB")



gc.collect()
torch.cuda.empty_cache()

print("allocated:", torch.cuda.memory_allocated()/1024**2, "MB")
print("reserved:",  torch.cuda.memory_reserved()/1024**2, "MB")

import logging
from ft_pipeline.logger import setup_logger
from ft_pipeline.config import FTConfig
from ft_pipeline.run_sft import run_finetune

CUDA available: True
CUDA device: NVIDIA A100-SXM4-40GB
allocated: 0.0 MB
reserved: 0.0 MB
allocated: 0.0 MB
reserved: 0.0 MB


In [2]:
# =========================
# Cell 2 — FTConfig 
# =========================
cfg = FTConfig(
    # -----------------------
    # PATHS / INPUT-OUTPUT
    # -----------------------
    model_id="/home/jovyan/ai-models/MamayLM-Gemma-3-12B",  # path or HF repo id base model
    train_jsonl="ft_datasets/sft_train.jsonl",             # train dataset in JSONL
    val_jsonl="ft_datasets/sft_val.jsonl",                 # validation dataset in JSONL
    out_dir="MamayLM-Gemma-3-12b_QLoRA_SFT",         

    # -----------------------
    # SEQUENCE / BATCHING
    # -----------------------
    max_seq_len=5000,                 # max context (promt+complections)
    per_device_train_batch_size=1,    # batch size на GPU 
    per_device_eval_batch_size=1,     # batch size на eval 
    gradient_accumulation_steps=8,    # (effective batch = batch_size * grad_accum)

    # -----------------------
    # TRAINING SCHEDULE / OPTIM
    # -----------------------
    learning_rate=0.0000008,          
    num_train_epochs=2,               # (if  max_steps are provided - will ignored)
    max_steps=None, #400 None         # use instead - num_train_epochs
    warmup_ratio=0.05,              
    lr_scheduler_type="cosine",       # scheduler: "cosine", "linear", ...
    logging_steps=5,                  # how often to log (steps)
    eval_steps=50,                    # how often to eval (steps)
    save_steps=200,                   # how often to save checkpoint (steps)
    save_total_limit=2,               # how many checkpoint to save
    weight_decay=0.01,                # L2 regularization 

    use_bf16=True,                    # BF16 (A100 — best)
    use_fp16=False,                   # fallback FP16 (if bf16 unavailable)

    # -----------------------
    # QLORA / BNB (4-bit quant)
    # -----------------------
    load_in_4bit=True,                # QLoRA 4-bit loading
    bnb_4bit_quant_type="nf4",        # quantization (nf4 — standart)
    bnb_4bit_use_double_quant=True,   # double-quant (often is good for quolity)
    attn_implementation="sdpa",       # "sdpa" — stable; 

    # -----------------------
    # LORA (adapter)
    # -----------------------
    lora_r=16,                        # rank
    lora_alpha=32,                    # scaling (often 2*r or 4*r for SFT)
    lora_dropout=0.05,                # dropout in LoRA
    target_modules=None,              # None → default in resolved_target_modules()

    # -----------------------
    # TRAINER BEHAVIOR
    # -----------------------
    packing=False,                    # packing a few samples in one seq 
    optim="paged_adamw_8bit",         # optimizator (8bit AdamW з bitsandbytes)
    report_to="none",                 # "none", "wandb", ...

    # -----------------------
    # INFERENCE SANITY CHECKS
    # -----------------------
    max_new_tokens_eval=512,          #  in A/B sanity (before/after)
)

print(cfg)
setup_logger(level=logging.INFO, log_file=f"{cfg.out_dir}/ft_run_sft.log")

FTConfig(model_id='/home/jovyan/ai-models/MamayLM-Gemma-3-12B', train_jsonl='ft_datasets/sft_train.jsonl', val_jsonl='ft_datasets/sft_val.jsonl', out_dir='MamayLM-Gemma-3-12b_QLoRA_SFT', max_seq_len=5000, per_device_train_batch_size=1, per_device_eval_batch_size=1, gradient_accumulation_steps=8, learning_rate=8e-07, weight_decay=0.01, num_train_epochs=2, max_steps=None, warmup_ratio=0.05, lr_scheduler_type='cosine', logging_steps=5, eval_steps=50, save_steps=200, save_total_limit=2, use_bf16=True, use_fp16=False, load_in_4bit=True, bnb_4bit_quant_type='nf4', bnb_4bit_use_double_quant=True, attn_implementation='sdpa', lora_r=16, lora_alpha=32, lora_dropout=0.05, target_modules=None, packing=False, optim='paged_adamw_8bit', report_to='none', max_new_tokens_eval=512)


<Logger ft_pipeline (INFO)>

In [ ]:
# =========================
# Cell 3 — Run fine-tune (усі параметри run_finetune + коментарі)
# =========================

# run_finetune(cfg, ...) — entrypoint, which:
#   - load dadaset from prepared JSONL
#   - Load tokenizer+model (QLoRA 4-bit) + LoRA
#   - A/B BEFORE (optional)
#   - Build trainer ( completion-only loss throgh  masking collator)
#   - add callbacks 
#   - train
#   - save lora_adapter + tokenizer
#   - A/B AFTER + ab_report (optional)


sft_artifacts = run_finetune(
    cfg,
    ab_indices = None,
    # ab_indices = list(range(1)),# indexes from val_jsonl to use in A/B "before/after" (strict JSON parse rate )
    # ab_indices=[0, 1, 10, 25, 50,  100, 150,  200, 250, 300,],   
    do_ab_before=False,    #  True → will generate ab_before.json 
    do_ab_after=True,     #  True → will generate ab_after.json and make ab_report.md
    dataset_limits=(None, None),  # (train_limit, val_limit) (500, 100)   # None → full dataset.
    dataset_mode="prompt_completion", # only it for now
    clean_cuda_cache_before=True, #  True → before start will do: gc.collect() + torch.cuda.empty_cache()
)


12:58:30 | INFO    | === FT RUN START ===
12:58:30 | INFO    | CUDA available=True
12:58:30 | INFO    | CUDA device=NVIDIA A100-SXM4-40GB
12:58:30 | INFO    | Loading datasets
12:58:30 | INFO    |   train: ft_datasets/sft_train.jsonl
12:58:30 | INFO    |   val:   ft_datasets/sft_val.jsonl
12:58:55 | INFO    | Converting to prompt/completion format
12:58:55 | INFO    | Dataset ready | train=3059 | val=340
12:58:55 | INFO    | Loading tokenizer: /home/jovyan/ai-models/MamayLM-Gemma-3-12B
12:58:56 | INFO    | Tokenizer loaded
12:58:56 | INFO    | Loading base model (QLoRA)
12:58:56 | INFO    |   model_id: /home/jovyan/ai-models/MamayLM-Gemma-3-12B
12:58:56 | INFO    |   dtype: torch.bfloat16
12:58:56 | INFO    |   4bit: True
12:58:56 | INFO    |   attn_implementation: sdpa


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

13:00:05 | INFO    | Base model loaded
13:00:05 | INFO    | Enabling gradient checkpointing
13:00:05 | INFO    | Applying LoRA
13:00:05 | INFO    |   r=16, alpha=32, dropout=0.05
13:00:05 | INFO    |   target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj', 'up_proj', 'down_proj', 'gate_proj']
13:00:06 | INFO    | LoRA applied successfully
13:00:06 | INFO    | Trainable parameters:
trainable params: 68,456,448 || all params: 12,255,781,488 || trainable%: 0.5586
13:00:06 | INFO    | Building SFTConfig
13:00:06 | INFO    |   max_seq_len=5000
13:00:06 | INFO    |   batch_size=1
13:00:06 | INFO    |   grad_accum=8
13:00:06 | INFO    |   lr=8e-07
13:00:06 | INFO    | Building SFTTrainer
13:00:06 | INFO    |   train_samples=3059
13:00:06 | INFO    |   val_samples=340
13:00:06 | INFO    |   dataset_mode=prompt_completion
13:00:06 | INFO    |   max_seq_length=5000


Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Using auto half precision backend
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


13:00:07 | INFO    | labels shape: (1, 3136)
13:00:07 | INFO    | non -100 labels: 261
13:00:07 | INFO    | Starting training…


...


***** Running Evaluation *****
  Num examples = 340
  Batch size = 1


19:59:40 | INFO    | [step 450] eval_loss=0.0854 | train_tok=seq_mean:2576 prompt_mean:2284 loss_mean:292 loss_max:292 | gpu_mem(GB)=alloc:9.49 res:37.84 max_alloc:23.60 max_res:38.98 | elapsed=419.1m
19:59:40 | INFO    | EarlyStop(metric=eval_loss): improved from 0.093778 to 0.085384
20:03:34 | INFO    | [step 455] train_loss=0.6146 | lr=3.11714e-07 | grad_norm=5.7288 | train_tok=seq_mean:2914 prompt_mean:2610 loss_mean:310 loss_max:310 | gpu_mem(GB)=alloc:9.49 res:25.59 max_alloc:23.60 max_res:38.98 | elapsed=423.0m
20:07:27 | INFO    | [step 460] train_loss=0.8082 | lr=3.03306e-07 | grad_norm=6.7957 | train_tok=seq_mean:3116 prompt_mean:2874 loss_mean:246 loss_max:246 | gpu_mem(GB)=alloc:9.49 res:15.71 max_alloc:23.60 max_res:38.98 | elapsed=426.9m
20:11:16 | INFO    | [step 465] train_loss=0.6033 | lr=2.94943e-07 | grad_norm=6.2068 | train_tok=seq_mean:3123 prompt_mean:2837 loss_mean:291 loss_max:291 | gpu_mem(GB)=alloc:9.49 res:11.69 max_alloc:23.60 max_res:38.98 | elapsed=430.7m



***** Running Evaluation *****
  Num examples = 340
  Batch size = 1


20:46:25 | INFO    | [step 500] eval_loss=0.0796 | train_tok=seq_mean:2576 prompt_mean:2284 loss_mean:292 loss_max:292 | gpu_mem(GB)=alloc:9.49 res:37.84 max_alloc:23.60 max_res:38.98 | elapsed=465.8m
20:46:25 | INFO    | EarlyStop(metric=eval_loss): improved from 0.085384 to 0.079573
20:50:04 | INFO    | [step 505] train_loss=0.5752 | lr=2.30126e-07 | grad_norm=7.0757 | train_tok=seq_mean:3165 prompt_mean:2885 loss_mean:283 loss_max:283 | gpu_mem(GB)=alloc:9.49 res:28.74 max_alloc:23.60 max_res:38.98 | elapsed=469.5m
20:53:54 | INFO    | [step 510] train_loss=0.6461 | lr=2.22342e-07 | grad_norm=5.0928 | train_tok=seq_mean:3092 prompt_mean:2846 loss_mean:250 loss_max:250 | gpu_mem(GB)=alloc:9.49 res:18.21 max_alloc:23.60 max_res:38.98 | elapsed=473.3m
20:57:45 | INFO    | [step 515] train_loss=0.5249 | lr=2.14641e-07 | grad_norm=5.5925 | train_tok=seq_mean:2771 prompt_mean:2462 loss_mean:314 loss_max:314 | gpu_mem(GB)=alloc:9.49 res:38.21 max_alloc:23.60 max_res:38.98 | elapsed=477.1m



***** Running Evaluation *****
  Num examples = 340
  Batch size = 1


In [1]:
sft_artifacts

NameError: name 'sft_artifacts' is not defined

# 🧠 Fine-tuning Mamay12B: SFT + DPO — Practical Guide
#### QLoRA — Quantized Low-Rank Adaptation

This guide describes **two training stages** (SFT → DPO) for a tariff recommendation assistant  
and **what to monitor in metrics + which parameters to control**.

---

## 1️⃣ SFT — Supervised Fine-Tuning

### 🎯 Goal
- teach the model a **stable JSON format**
- correct usage analysis
- proper language and response structure  
> **SFT does not optimize tariff selection**, only behavior and formatting.

---

### 📊 Key metrics (monitor in logs)

#### 🔹 `train_loss`
- expected: **smoothly decreases**
- typical range: `~1.5 → 0.5–0.8`
- ❌ bad: sharp drop to `~0.0` → overfitting

#### 🔹 `eval_loss`
- should **correlate** with `train_loss`
- ❌ if `train ↓` while `eval ↑` → overfitting

#### 🔹 A/B sanity (before / after)
(via `ABSanityCallback`)
- JSON parses in **100% of cases**
- all required fields are present
- `tariffId ∈ avail_tp_with_desc`
- text is in Ukrainian

---

### 🎛️ Main knobs (SFT)

| Symptom | What to change |
|------|----------|
| loss does not decrease | ↑ `learning_rate` (5e-5 → 1e-4) |
| fast overfitting | ↓ `learning_rate`, ↓ `num_train_epochs` |
| brittle JSON | ↑ dataset size, ↓ LR |
| slow training | ↓ `max_seq_len`, ↑ `grad_accum` |

---


## SFT metrics: what they mean & how to tune them (Tariff Recommender)

This notebook runs **SFT (Supervised Fine-Tuning)** with **completion-only loss**:
- We feed a long `prompt` (facts + question + formatting rules)
- We train the model to generate the `completion` (assistant JSON)
- **Loss is computed only on completion tokens** (prompt tokens are masked out)

SFT teaches:
- stable output format (strict JSON)
- the correct content structure (fields, Ukrainian text sections)
- general mapping from usage → recommendation patterns (but not pairwise ranking like DPO)

---

### Key logged metrics (what they mean)

#### 1) `train_loss`
- Cross-entropy loss on **completion tokens only**.
- Lower is better, but:
  - very low train_loss can mean **overfitting** (especially on small datasets)
  - always compare with `eval_loss`

**If train_loss drops fast but eval_loss stalls or rises:**
- overfitting → reduce steps/epochs, reduce LR, add regularization (weight_decay, dropout), or stop early

---

#### 2) `eval_loss`
- Cross-entropy loss on validation set completion tokens.
- This is your primary “generalization” signal.

Interpretation:
- **decreasing** eval_loss → model generalizes better
- **flat** eval_loss → you’re near the best point
- **increasing** eval_loss → overfitting (stop / revert to best checkpoint)

✅ Typical workflow:
- Use early stopping on `eval_loss`
- Keep the checkpoint with the **lowest** eval_loss

---

#### 3) `learning_rate` (LR schedule)
- LR warms up and then follows your scheduler (cosine/linear).
- LR that is too high can:
  - make training unstable
  - harm format stability (JSON breaks)
- LR too low can:
  - learn very slowly / plateau early

**Recommended starting LR for your setup (12B + QLoRA + long prompts):**
- `learning_rate = 5e-5` (`0.00005`)
- avoid `1e-4` unless you have a large, diverse dataset and see stable format metrics

---

#### 4) `grad_norm`
- How “big” the update is at each step (stability indicator).
- Useful for catching too-aggressive training before quality degrades.

Rules of thumb:
- stable regime: `~1–10` (depends on setup)
- frequent spikes `>15–20` → LR too high or batch noise → reduce LR / clip gradients

---

#### 5) Token stats: `train_tok=...`
Example:
- `seq_mean`: average total tokens fed to the model
- `prompt_mean`: average prompt tokens (masked from loss)
- `loss_mean`: average tokens contributing to loss (completion length)
- `loss_max`: max completion length seen in the batch

Why it matters:
- if `loss_mean` becomes tiny (e.g., 20–50) you’re barely training on the answer
- if `seq_mean` is close to `max_seq_len`, you risk truncation

✅ In your case:
- prompt ~2800–2900, completion ~250–350
- ensure `max_seq_len=4096` to avoid cutting prompt or answer

---

#### 6) Format/quality sanity metric: `valid_json_rate` (from A/B sanity)
- % of sanity examples where the generated output is valid JSON (and parseable).
- This is **production-critical** for your tariff assistant.

Interpretation:
- `100%` → stable format
- dips (e.g., `80%`) → likely decoding randomness or format instability

✅ For reliable tracking, run sanity with deterministic generation:
- `do_sample=False`, `num_beams=1`
- explicitly disable sampling params (temperature/top_p/top_k)

---

### What to tune (control knobs)

#### A) `learning_rate` (most important)
Safe defaults for your setup:
- `learning_rate = 5e-5` (`0.00005`)
If you see instability / JSON breaks:
- reduce to `3e-5` (`0.00003`)
If you learn too slowly:
- increase slightly, but prefer more steps over large LR

Symptoms → Fix:
- `grad_norm` spikes, `train_loss` noisy → lower LR
- `valid_json_rate` drops → lower LR + deterministic eval + maybe reduce steps

---

#### B) `max_steps` / `num_train_epochs`
SFT often converges quickly on structured outputs.
Use either:
- fixed `max_steps` (best for reproducibility)
- or `num_train_epochs` (less predictable if dataset changes)

For ~3k rows, `batch=1`, `grad_accum=8`:
- 1 epoch ≈ ~`3094/8 ≈ 387` optimizer steps

Practical:
- start with `max_steps = 400–800` (≈ 1–2 epochs)
- rely on early stopping to stop before overfitting

---

#### C) `eval_steps` (how often to evaluate)
Trade-off:
- frequent eval = better early stopping decisions but slower runs

Practical:
- `eval_steps = 100` (good default)
- if eval is expensive, `eval_steps = 200` is ok (you used 200)

---

#### D) `save_steps` and `save_total_limit`
Keep enough checkpoints to recover best eval point:
- `save_steps = eval_steps` (common)
- `save_total_limit = 2–3`

---

#### E) Regularization knobs
Useful when eval_loss stops improving while train_loss keeps falling:
- `weight_decay = 0.01` (you already added — good)
- `lora_dropout = 0.05` (good)
- reduce `lora_r` if needed (e.g., 16 → 8) for smaller capacity (rare)

---

#### F) Sequence length and truncation protection
Given your distribution (prompt p95 ~2880, answers ~300–350):
- use `max_seq_len = 4096`
- keep `max_new_tokens_eval` around `256–512`

Symptoms of truncation:
- worse eval_loss
- inconsistent content fields
- model ignores the tail of `avail_tp_with_desc`

---

### What to watch for in practice (quick checklist)

✅ Healthy SFT run:
- `train_loss` decreases steadily
- `eval_loss` decreases then plateaus
- `valid_json_rate` stays high (ideally 100% on deterministic sanity)
- generations remain consistent before/after checkpoints

⚠️ Overfitting signs:
- `train_loss` keeps dropping but `eval_loss` stops improving or rises
- outputs become overly rigid, repetitive, or “memorized”
- downstream DPO becomes too easy (margins explode fast)

**Fix overfitting:**
- fewer steps / fewer epochs
- lower `learning_rate`
- stronger regularization (weight_decay / dropout)
- early stopping on `eval_loss`

---

### Recommended production evaluation pattern
- **Full eval**: every `eval_steps` via `eval_loss`
- **Sanity subset**: run A/B sanity on a fixed set of indices
  - keep generation deterministic
  - track:
    - `valid_json_rate`
    - presence/validity of required fields (`tariffId`, `templateId`, etc.)
    - (optional) exact tariffId match rate on sanity subset

This combination prevents “loss looks good but output is broken” failures.
